In [ ]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint
m = wt.watertap_setup(dynamic=False)


wt.case_study_trains.train = {"case_study": "ocwd",
                             "reference": "nawi",
                             "scenario": "baseline"}

wt.case_study_trains.source_water = {"case_study": "ocwd", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "ocwd_feed"}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.

In [ ]:
wt.display.show_train2(model_name=m)

In [ ]:
### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')
        
wt.run_water_tap(m=m, objective=True)

# prints RO results
print("\nOptimal RO area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            if m.fs.pfd_dict[key]["Parameter"]["type"] == "pass":
                getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
                print(f'\nSetting feed presure for {key} to {m.fs.pfd_dict[key]["Parameter"]["feed_pressure"]} bar (user input)...\n')

wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

In [ ]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 1

In [ ]:
wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

In [ ]:
# RESET AREA TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [ ]:
# Readjust recovery constraint and deactivate objective constraint
m.recovery_bound = Constraint(expr=m.fs.costing.system_recovery >= 0)
m.fs.objective_function.deactivate()

In [ ]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(f'\tFeed pressure for {key}: {round(getattr(m.fs, key).feed.pressure[0](), 2)} bar')
        print(f'\tMembrane area for {key}: {round(getattr(m.fs, key).membrane_area[0](), 2)} m2')

In [ ]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario=wt.case_study_trains.source_water["scenario"])